<a href="https://colab.research.google.com/github/N-django/data-analysis-vehicle-Insuranceclaims/blob/main/Poisson_GLM_for_claim_Frequency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import pandas as pd


df = pd.read_excel('/content/drive/MyDrive/Motor Private Dataset.xlsx')


df.head()

,CMB_CLAIM_NO,POLICY_NO,EF,ET,LOSS_DATE,REPORT DATE,Vehicle_ID,YOM,AGE_CATEGORY,CAR_AGE,COLORS,CUBIC_CAPACITY,CARRY_CAPACITY,BODY TYPE,REG_Name,VEHICLE MAKE,VEHICLE MODELS,GROSS_PAID
0,C/10/700/043075,10/700/0030439,06/09/2023,06/08/2024,2024-05-13,2024-05-14,KCY 676R,2014.0,8+ Years,10.0,BLACK,2493.0,5.0,STATION WAGON,TOYOTA HARRIER HYBDRID,TOYOTA,TOYOTA HARRIER,0.0
1,C/12/700/041177,12/700/0071745,03/01/2023,02/29/2024,2024-01-26,2024-02-15,KCM 432W,2011.0,8+ Years,13.0,WHITE,2400.0,5.0,STATION WAGON,TOYOTA RAV4,TOYOTA,TOYOTA RAV4,0.0
2,C/40/700/037888,AE700/0065804/000/00,03/03/2023,01/24/2024,2023-08-30,2023-09-01,KDL 192F,2017.0,4-7 Years,7.0,SILVER,1496.0,5.0,STATION WAGON,SUBARU FORESTER,SUBARU,SUBARU FORESTER,0.0
3,C/10/700/036490,10/700/0101357,01/01/2023,12/31/2023,2023-04-14,2023-06-19,KBZ 107K,2007.0,8+ Years,17.0,SILVER,1496.0,5.0,SEDAN,SALOON AUTO,UNKNOWN,SALOON AUTO,0.0
4,C/15/700/033788,AI700/0060894/000/00,09/21/2022,07/20/2023,2023-01-23,2023-01-30,KCH 319U,2009.0,8+ Years,15.0,BLACK,2488.0,5.0,STATION WAGON,NISSAN XTRAIL,NISSAN,NISSAN X-TRAIL,0.0


In [1]:
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import re
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm


In [6]:
median_yom = df['YOM'].median()
df['YOM']= df['YOM'].fillna(median_yom)

mean_car_age = df['CAR_AGE'].mean()
mean_cubic_capacity = df['CUBIC_CAPACITY'].mean()
mean_carry_capacity = df['CARRY_CAPACITY'].mean()

df['CAR_AGE'] = df['CAR_AGE'].fillna(mean_car_age)
df['CUBIC_CAPACITY'] = df['CUBIC_CAPACITY'].fillna(mean_cubic_capacity)
df['CARRY_CAPACITY'] = df['CARRY_CAPACITY'].fillna(mean_carry_capacity)
df['REG_Name']=df['REG_Name'].fillna('UNKNOWN')
df['GROSS_PAID']=df['GROSS_PAID'].fillna(0)

Blank_columns = df.isna().sum()

df['Claim_Frequency'] = df.groupby('Vehicle_ID')['CMB_CLAIM_NO'].transform('count')
print(f"Total number of missing values: {Blank_columns}")

Total number of missing values: CMB_CLAIM_NO      0
POLICY_NO         0
EF                0
ET                0
LOSS_DATE         0
REPORT DATE       0
Vehicle_ID        0
YOM               0
AGE_CATEGORY      0
CAR_AGE           0
 COLORS           0
CUBIC_CAPACITY    0
CARRY_CAPACITY    0
BODY TYPE         0
REG_Name          0
VEHICLE MAKE      0
VEHICLE MODELS    0
GROSS_PAID        0
dtype: int64


In [ ]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 10.4 MB/s eta 0:00:00


In [10]:
df.columns



Index(['CMB_CLAIM_NO', 'POLICY_NO', 'EF', 'ET', 'LOSS_DATE', 'REPORT DATE',
       'Vehicle_ID', 'YOM', 'AGE_CATEGORY', 'CAR_AGE', ' COLORS',
       'CUBIC_CAPACITY', 'CARRY_CAPACITY', 'BODY TYPE', 'REG_Name',
       'VEHICLE MAKE', 'VEHICLE MODELS', 'GROSS_PAID', 'Claim_Frequency'],
      dtype='object')

In [11]:
!pip install category_encoders


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 3.7 MB/s eta 0:00:00


In [9]:
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split

categorical_cols = [' COLORS', 'BODY TYPE', 'VEHICLE MAKE', 'VEHICLE MODELS']
numerical_cols = ['CAR_AGE', 'CUBIC_CAPACITY', 'CARRY_CAPACITY']
target = 'Claim_Frequency'

# Target encode categorical variables
encoder = TargetEncoder(cols=categorical_cols)
df_encoded = encoder.fit_transform(df[categorical_cols], df[target])

# Combine numerical and encoded categorical features
X = pd.concat([df[numerical_cols], df_encoded], axis=1)

# Add a constant (intercept term) for the GLM
X = sm.add_constant(X)

# Target variable
y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit a GLM model with Poisson distribution
poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Print model summary
print(poisson_results.summary())

# Predict on test set
y_pred = poisson_results.predict(X_test)

# Evaluate predictions
print("Predicted values:", y_pred[:5])

ModuleNotFoundError: No module named 'category_encoders'

In [8]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from sklearn.linear_model import PoissonRegressor



# Define the feature and target variables
categorical_cols = [' COLORS', 'BODY TYPE', 'VEHICLE MAKE', 'VEHICLE MODELS']
numerical_cols = ['CAR_AGE', 'CUBIC_CAPACITY', 'CARRY_CAPACITY']
target = 'Claim_Frequency'

# Target encode categorical variables
encoder = TargetEncoder(cols=categorical_cols)
df_encoded = encoder.fit_transform(df[categorical_cols], df[target])

# Combine numerical and encoded categorical features
X = pd.concat([df[numerical_cols], df_encoded], axis=1)

# Impute missing values (if any) with SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Generate polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_imputed)

# Get feature names for the polynomial features
combined_feature_names = numerical_cols + list(df_encoded.columns)
poly_feature_names = poly.get_feature_names_out(combined_feature_names)
X_poly_df = pd.DataFrame(X_poly, columns=poly_feature_names)

# Add a constant (intercept term) for the GLM
X_poly_df = sm.add_constant(X_poly_df)

# Target variable
y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_poly_df, y, test_size=0.2, random_state=42)

# Fit a GLM model with Poisson distribution
poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Print model summary
print(poisson_results.summary())

# Predict on the test set
y_pred = poisson_results.predict(X_test)

# Evaluate predictions (you can choose different evaluation metrics here)
print("Predicted values (first 5):", y_pred[:5])

ModuleNotFoundError: No module named 'category_encoders'

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import PoissonRegressor  # GLM Poisson regressor
import statsmodels.api as sm

# Categorical and numerical columns
categorical_cols = [' COLORS', 'BODY TYPE', 'VEHICLE MAKE', 'VEHICLE MODELS']
numerical_cols = ['CAR_AGE', 'CUBIC_CAPACITY', 'CARRY_CAPACITY']
target = 'Claim_Frequency'

# Target encode categorical variables
encoder = TargetEncoder(cols=categorical_cols)
df_encoded = encoder.fit_transform(df[categorical_cols], df[target])

# Combine numerical and encoded categorical features
X = pd.concat([df[numerical_cols], df_encoded], axis=1)

# Impute missing values (if any) with SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Generate polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_imputed)

# Get feature names for the polynomial features
combined_feature_names = numerical_cols + list(df_encoded.columns)
poly_feature_names = poly.get_feature_names_out(combined_feature_names)
X_poly_df = pd.DataFrame(X_poly, columns=poly_feature_names)

# Add a constant (intercept term) for the GLM
X_poly_df = sm.add_constant(X_poly_df)

# Target variable
y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_poly_df, y, test_size=0.2, random_state=42)

# Initialize a Poisson Regressor for RFE
poisson_reg = PoissonRegressor(alpha=0.0, fit_intercept=False, max_iter=1000)

# Apply RFE (Recursive Feature Elimination) for feature selection
selector = RFE(estimator=poisson_reg, n_features_to_select=50, step=1)  # Select top 25 features
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Print selected features
selected_features = X_train.columns[selector.support_]
print("Selected Features:", selected_features)

# Create a mapping of selected features to their coefficients
feature_mapping = {f'x{i+1}': feature for i, feature in enumerate(selected_features)}

# Fit a GLM model with Poisson distribution using selected features
poisson_model = sm.GLM(y_train, X_train_selected, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Print model summary
print(poisson_results.summary())

# Print the mapping of selected features to coefficients
print("Feature to Coefficient Mapping:")
for key, value in feature_mapping.items():
    print(f"{key}: {value}")

# Predict on the test set using selected features
y_pred = poisson_results.predict(X_test_selected)

# Evaluate predictions
print("Predicted values (first 5):", y_pred[:5])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=50 > n_features=36. There will be no feature selection and all features will be kept.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_linear_loss.py:295: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights


Selected Features: Index(['const', 'CAR_AGE', 'CUBIC_CAPACITY', 'CARRY_CAPACITY', ' COLORS',
       'BODY TYPE', 'VEHICLE MAKE', 'VEHICLE MODELS', 'CAR_AGE^2',
       'CAR_AGE CUBIC_CAPACITY', 'CAR_AGE CARRY_CAPACITY', 'CAR_AGE  COLORS',
       'CAR_AGE BODY TYPE', 'CAR_AGE VEHICLE MAKE', 'CAR_AGE VEHICLE MODELS',
       'CUBIC_CAPACITY^2', 'CUBIC_CAPACITY CARRY_CAPACITY',
       'CUBIC_CAPACITY  COLORS', 'CUBIC_CAPACITY BODY TYPE',
       'CUBIC_CAPACITY VEHICLE MAKE', 'CUBIC_CAPACITY VEHICLE MODELS',
       'CARRY_CAPACITY^2', 'CARRY_CAPACITY  COLORS',
       'CARRY_CAPACITY BODY TYPE', 'CARRY_CAPACITY VEHICLE MAKE',
       'CARRY_CAPACITY VEHICLE MODELS', ' COLORS^2', ' COLORS BODY TYPE',
       ' COLORS VEHICLE MAKE', ' COLORS VEHICLE MODELS', 'BODY TYPE^2',
       'BODY TYPE VEHICLE MAKE', 'BODY TYPE VEHICLE MODELS', 'VEHICLE MAKE^2',
       'VEHICLE MAKE VEHICLE MODELS', 'VEHICLE MODELS^2'],
      dtype='object')
                 Generalized Linear Model Regression Results       

In [ ]:
print(df.isna().sum())

CMB_CLAIM_NO       0
POLICY_NO          0
EF                 0
ET                 0
LOSS_DATE          0
REPORT DATE        0
Vehicle_ID         0
YOM                0
AGE_CATEGORY       0
CAR_AGE            0
 COLORS            0
CUBIC_CAPACITY     0
CARRY_CAPACITY     0
BODY TYPE          0
REG_Name           0
VEHICLE MAKE       0
VEHICLE MODELS     0
GROSS_PAID         0
Claim_Frequency    0
dtype: int64


In [ ]:
feature_mapping = list(poly_feature_names)
print(feature_mapping)

['CAR_AGE', 'CUBIC_CAPACITY', 'CARRY_CAPACITY', ' COLORS', 'BODY TYPE', 'VEHICLE MAKE', 'VEHICLE MODELS', 'CAR_AGE^2', 'CAR_AGE CUBIC_CAPACITY', 'CAR_AGE CARRY_CAPACITY', 'CAR_AGE  COLORS', 'CAR_AGE BODY TYPE', 'CAR_AGE VEHICLE MAKE', 'CAR_AGE VEHICLE MODELS', 'CUBIC_CAPACITY^2', 'CUBIC_CAPACITY CARRY_CAPACITY', 'CUBIC_CAPACITY  COLORS', 'CUBIC_CAPACITY BODY TYPE', 'CUBIC_CAPACITY VEHICLE MAKE', 'CUBIC_CAPACITY VEHICLE MODELS', 'CARRY_CAPACITY^2', 'CARRY_CAPACITY  COLORS', 'CARRY_CAPACITY BODY TYPE', 'CARRY_CAPACITY VEHICLE MAKE', 'CARRY_CAPACITY VEHICLE MODELS', ' COLORS^2', ' COLORS BODY TYPE', ' COLORS VEHICLE MAKE', ' COLORS VEHICLE MODELS', 'BODY TYPE^2', 'BODY TYPE VEHICLE MAKE', 'BODY TYPE VEHICLE MODELS', 'VEHICLE MAKE^2', 'VEHICLE MAKE VEHICLE MODELS', 'VEHICLE MODELS^2']


In [7]:
categorical_cols = [' COLORS', 'BODY TYPE', 'VEHICLE MAKE', 'VEHICLE MODELS']
numerical_cols = ['CAR_AGE', 'CUBIC_CAPACITY', 'CARRY_CAPACITY']
target = 'Claim_Frequency'

# Target encode categorical variables
encoder = TargetEncoder(cols=categorical_cols)
df_encoded = encoder.fit_transform(df[categorical_cols], df[target])

# Combine numerical and encoded categorical features
X = pd.concat([df[numerical_cols], df_encoded], axis=1)

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Generate polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_imputed)

# Get feature names for the polynomial features
combined_feature_names = numerical_cols + list(df_encoded.columns)
poly_feature_names = poly.get_feature_names_out(combined_feature_names)
X_poly_df = pd.DataFrame(X_poly, columns=poly_feature_names)

# Add a constant for the GLM
X_poly_df = sm.add_constant(X_poly_df)

# Target variable
y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_poly_df, y, test_size=0.2, random_state=42)

# Initialize a Poisson Regressor for RFE
poisson_reg = PoissonRegressor(alpha=0.0, fit_intercept=False, max_iter=1000)

# Apply RFE for feature selection
selector = RFE(estimator=poisson_reg, n_features_to_select=50, step=1)  # Select top 25 features
X_train_selected = selector.fit_transform(X_train, y_train)

# Define the test set selected features after RFE
X_test_selected = selector.transform(X_test)

# Retain original variables
original_vars = X_train[numerical_cols].reset_index(drop=True)

# Convert selected features back to DataFrame
X_train_selected_df = pd.DataFrame(X_train_selected, columns=X_train.columns[selector.support_]).reset_index(drop=True)
X_test_selected_df = pd.DataFrame(X_test_selected, columns=X_test.columns[selector.support_]).reset_index(drop=True)

# Combine original numerical variables with selected polynomial features
X_train_final = pd.concat([original_vars, X_train_selected_df], axis=1)
X_test_final = pd.concat([X_test[numerical_cols].reset_index(drop=True), X_test_selected_df.reset_index(drop=True)], axis=1)

# Ensure y_train has the same index as X_train_final
y_train_final = y_train.reset_index(drop=True)

# Fit a GLM model with Poisson distribution using final selected features
poisson_model = sm.GLM(y_train_final, X_train_final, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Print model summary
print(poisson_results.summary())

# Predict on the test set using final selected features
y_pred = poisson_results.predict(X_test_final)

# Evaluate predictions
print("Predicted values (first 5):", y_pred[:5])

NameError: name 'TargetEncoder' is not defined

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Convert final training and testing sets to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train_final, label=y_train_final)
dtest = xgb.DMatrix(X_test_final, label=y_test)

# Define XGBoost parameters
xgb_params = {
    'objective': 'count:poisson',  # Poisson regression for claim frequency
    'max_depth': 6,                # Maximum tree depth
    'eta': 0.1,                    # Learning rate
    'subsample': 0.8,              # Row sampling
    'colsample_bytree': 0.8,       # Feature sampling
    'eval_metric': 'rmse'          # Root mean square error
}

# Train XGBoost model
xgb_model = xgb.train(params=xgb_params, dtrain=dtrain, num_boost_round=100)

# Predict on the test set
y_pred_xgb = xgb_model.predict(dtest)

# Evaluate the XGBoost model predictions
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"XGBoost Mean Squared Error: {mse_xgb}")
print(f"XGBoost R-squared: {r2_xgb}")

# Feature importance from XGBoost
xgb_feature_importance = xgb_model.get_score(importance_type='weight')

# Sort and display the top features based on importance
sorted_importance = sorted(xgb_feature_importance.items(), key=lambda x: x[1], reverse=True)
print("Top 10 XGBoost Features by Importance:")
for feature, importance in sorted_importance[:10]:
    print(f"{feature}: {importance}")